In [3]:
# Q1. Running Elastic

In [5]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [82]:
# documents

In [7]:
# Q2. Indexing the data

In [8]:
from elasticsearch import Elasticsearch

In [9]:
es_client = Elasticsearch('http://localhost:9200')

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [11]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 42.22it/s]


In [13]:
# Q3. Searching

In [37]:
def elastic_search_scores(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    scores = []
    
    for hit in response['hits']['hits']:
        scores.append(hit['_score'])
    
    return scores

In [81]:
query = "How do I execute a command in a running docker container?"
elastic_search_scores(query)[0]

84.050095

In [39]:
# Q4. Filtering

In [40]:
def elastic_search_filtering(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [44]:
query = "How do I execute a command in a running docker container?"
elastic_search_filtering(query)[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [45]:
# Q5. Building a prompt

In [72]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [73]:
query = "How do I execute a command in a running docker container?"
len(build_prompt(query, elastic_search_filtering(query)))

1462

In [ ]:
# Q6. Tokens

In [76]:
import tiktoken

In [78]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [80]:
len(encoding.encode(build_prompt(query, elastic_search_filtering(query))))

322